In [1]:
import os
import glob
import sys

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from ragas.testset import TestsetGenerator
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

c:\Users\safak\Desktop\devs\git_repos\search-systems-self\my_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [45]:
docs = []
for file in os.listdir("./"):
    if file.endswith("test.pdf"):
        loader = PyPDFLoader(f"./{file}")
        docs.extend(loader.load())

In [38]:
generator_llm = ChatOpenAI(model_name="gpt-4")
judge_llm = ChatOpenAI(model_name="gpt-4")
embeddings = OpenAIEmbeddings()

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 1. Initialize LangChain objects
lang_llm = ChatOpenAI(model="gpt-4o")
lang_embeddings = OpenAIEmbeddings()

# 2. Wrap them for Ragas compatibility
# This adds the .embed_text() method that Ragas is looking for
generator_llm = LangchainLLMWrapper(lang_llm)
generator_embeddings = LangchainEmbeddingsWrapper(lang_embeddings)

# 3. Initialize the generator with the WRAPPED versions
from ragas.testset import TestsetGenerator
generator = TestsetGenerator(
    llm=generator_llm, 
    embedding_model=generator_embeddings
)

# 4. If you are using custom transforms, they must also use the wrapped embeddings
from ragas.testset.transforms import default_transforms
transforms = default_transforms(documents=docs, llm=generator_llm, embedding_model=generator_embeddings)

# 5. Run
test_set = generator.generate_with_langchain_docs(docs, testset_size=2, transforms=transforms)

In [58]:
test_set_df = test_set.to_pandas()

In [59]:
test_set_df

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,Wht role did Safak Atakan Celik play in Street...,[Safak Atakan Celik\n(data analyst)\n (778) 95...,Safak Atakan Celik served as the Data Processi...,Data Analytics Professional,MISSPELLED,LONG,single_hop_specific_query_synthesizer
1,What kind of solutions did Streetbees provide ...,[Safak Atakan Celik\n(data analyst)\n (778) 95...,Streetbees delivered end-to-end solutions that...,Data Analytics Professional,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer


In [92]:
# 1. Your RAG system logic (simulated here)
# Important: ensure you process the questions in the EXACT order of the test_set_df
rag_answers = ["Paris is the capital.", "Transformers work in parallel."]
rag_contexts = [
    ["Paris is the capital of France."], 
    ["Transformers use self-attention to process data in parallel."]
]

# 2. Build the combined DataFrame
eval_df = pd.DataFrame({
    "question": test_set_df["user_input"].tolist(),
    "ground_truth": test_set_df["reference"].tolist(),
    "answer": rag_answers,
    "contexts": rag_contexts
})

In [93]:
eval_df

,question,ground_truth,answer,contexts
0,Wht role did Safak Atakan Celik play in Street...,Safak Atakan Celik served as the Data Processi...,Paris is the capital.,[Paris is the capital of France.]
1,What kind of solutions did Streetbees provide ...,Streetbees delivered end-to-end solutions that...,Transformers work in parallel.,[Transformers use self-attention to process da...


---

ollama


In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_ollama import ChatOllama, OllamaEmbeddings
from ragas.testset import TestsetGenerator
from langchain_community.document_loaders import PyPDFLoader

In [4]:
# 1. Initialize the Qwen LLM and Embeddings from Ollama
# Qwen 2.5 7B is highly recommended for this
local_llm = ChatOllama(model="qwen2.5:7b")
local_embeddings = OllamaEmbeddings(model="nomic-embed-text") # Or "qwen2.5:7b" if preferred

In [5]:
# 2. Wrap them for Ragas compatibility
generator_llm = LangchainLLMWrapper(local_llm)
critic_llm = LangchainLLMWrapper(local_llm)
embeddings = LangchainEmbeddingsWrapper(local_embeddings)

C:\Users\safak\AppData\Local\Temp\ipykernel_13600\2254106299.py:2: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(local_llm)
C:\Users\safak\AppData\Local\Temp\ipykernel_13600\2254106299.py:3: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  critic_llm = LangchainLLMWrapper(local_llm)
C:\Users\safak\AppData\Local\Temp\ipykernel_13600\2254106299.py:4: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client

In [8]:
# 3. Initialize the Generator
generator = TestsetGenerator(
    generator_llm,
    critic_llm,
    embeddings
)

In [10]:
docs = []
for file in os.listdir("./"):
    if file.endswith("test.pdf"):
        loader = PyPDFLoader(f"./{file}")
        docs.extend(loader.load())

In [15]:
from ragas.run_config import RunConfig
# 4. Generate the testset (assuming 'docs' is your Langchain document list)
dataset = generator.generate_with_langchain_docs(docs, testset_size=2, run_config=RunConfig(max_workers=1))

Applying HeadlinesExtractor: 100%|██████████| 1/1 [00:39<00:00, 39.18s/it]


ResponseError: model requires more system memory (5.6 GiB) than is available (4.5 GiB) (status code: 500)